In [65]:
#warnings
import warnings
warnings.filterwarnings('ignore')

# modules
import pandas as pd
import numpy as np
import acquire
import prepare
import wrangle
import explore
import evaluate
import pydataset
from pydataset import data
import feature_engineering 

#math/ visualization
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
from scipy import stats
from sklearn.model_selection import train_test_split
import sklearn.preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.feature_selection import f_regression, RFE, SelectKBest
import sklearn.preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression 


In [ ]:
#insert helper function here

Do your work for this exercise in a jupyter notebook named feature_engineering within the regression-exercises repo. Add, commit, and push your work.

### **1. Load the tips dataset.**

   

In [ ]:
# .size property rows * columsn

In [ ]:
# rename size. columns to avoid future confusion

In [66]:
tips = data("tips")

In [67]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


In [68]:
tips.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [69]:
tips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244 entries, 1 to 244
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  244 non-null    float64
 1   tip         244 non-null    float64
 2   sex         244 non-null    object 
 3   smoker      244 non-null    object 
 4   day         244 non-null    object 
 5   time        244 non-null    object 
 6   size        244 non-null    int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 15.2+ KB


In [70]:
tips = tips.drop(columns = ['sex', 'smoker', 'day', 'time'])

In [71]:
tips.shape

(244, 3)

### **a. Create a column named tip_percentage. This should be the tip amount divided by the total bill.**
       

In [72]:
tips['tip_percentage'] = tips.tip / tips.total_bill


In [73]:
tips.head()

,total_bill,tip,size,tip_percentage
1,16.99,1.01,2,0.059447
2,10.34,1.66,3,0.160542
3,21.01,3.50,3,0.166587
4,23.68,3.31,2,0.139780
5,24.59,3.61,4,0.146808


### **b. Create a column named price_per_person. This should be the total bill divided by the party size**

In [74]:
tips['price_per_person'] = tips.total_bill / tips.size

In [75]:
tips

,total_bill,tip,size,tip_percentage,price_per_person
1,16.99,1.01,2,0.059447,0.017408
2,10.34,1.66,3,0.160542,0.010594
3,21.01,3.50,3,0.166587,0.021527
4,23.68,3.31,2,0.139780,0.024262
5,24.59,3.61,4,0.146808,0.025195
...,...,...,...,...,...
240,29.03,5.92,3,0.203927,0.029744
241,27.18,2.00,2,0.073584,0.027848
242,22.67,2.00,2,0.088222,0.023227
243,17.82,1.75,2,0.098204,0.018258


In [76]:
#Data Prep
tips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244 entries, 1 to 244
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   total_bill        244 non-null    float64
 1   tip               244 non-null    float64
 2   size              244 non-null    int64  
 3   tip_percentage    244 non-null    float64
 4   price_per_person  244 non-null    float64
dtypes: float64(4), int64(1)
memory usage: 11.4 KB


### **c. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?**
   **- The tip percentage?**
       

### **d. Use all the other numeric features to predict tip amount. Use select k best and recursive feature elimination to select the top 2 features.**
    **- What are they?**
       

In [77]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
def train_validate_test(df, target):
    '''
    this function takes in a dataframe and splits it into 3 samples, 
    a test, which is 20% of the entire dataframe, 
    a validate, which is 24% of the entire dataframe,
    and a train, which is 56% of the entire dataframe. 
    It then splits each of the 3 samples into a dataframe with independent variables
    and a series with the dependent, or target variable. 
    The function returns 3 dataframes and 3 series:
    X_train (df) & y_train (series), X_validate & y_validate, X_test & y_test. 
    '''
    # split df into test (20%) and train_validate (80%)
    train_validate, test = train_test_split(df, test_size=.2, random_state=123)

    # split train_validate off into train (70% of 80% = 56%) and validate (30% of 80% = 24%)
    train, validate = train_test_split(train_validate, test_size=.3, random_state=123)

        
    # split train into X (dataframe, drop target) & y (series, keep target only)
    X_train = train.drop(columns=[target])
    y_train = train[target]
    
    # split validate into X (dataframe, drop target) & y (series, keep target only)
    X_validate = validate.drop(columns=[target])
    y_validate = validate[target]
    
    # split test into X (dataframe, drop target) & y (series, keep target only)
    X_test = test.drop(columns=[target])
    y_test = test[target]
    
    return X_train, y_train, X_validate, y_validate, X_test, y_test

In [78]:
X_train, y_train, X_validate, y_validate, X_test, y_test = train_validate_test(tips, 'tip' )
X_train.head()

,total_bill,size,tip_percentage,price_per_person
19,16.97,3,0.206246,0.017387
173,7.25,2,0.710345,0.007428
119,12.43,2,0.144811,0.012736
29,21.70,2,0.198157,0.022234
238,32.83,2,0.035638,0.033637


In [79]:
X_train, y_train, X_validate, y_validate, X_test, y_test = feature_engineering.train_validate_test(tips, target)

In [80]:

X_train_encoded = pd.get_dummies(X_train, columns = object_cols, drop_first=True)
X_train_encoded

,total_bill,size,tip_percentage,price_per_person
19,16.97,3,0.206246,0.017387
173,7.25,2,0.710345,0.007428
119,12.43,2,0.144811,0.012736
29,21.70,2,0.198157,0.022234
238,32.83,2,0.035638,0.033637
208,38.73,4,0.077459,0.039682
184,23.17,4,0.280535,0.023740
61,20.29,2,0.158206,0.020789
42,17.46,2,0.145475,0.017889
161,21.50,4,0.162791,0.022029


In [81]:
X_train.drop(columns=object_cols,inplace=True)
X_train

,total_bill,size,tip_percentage,price_per_person
19,16.97,3,0.206246,0.017387
173,7.25,2,0.710345,0.007428
119,12.43,2,0.144811,0.012736
29,21.70,2,0.198157,0.022234
238,32.83,2,0.035638,0.033637
208,38.73,4,0.077459,0.039682
184,23.17,4,0.280535,0.023740
61,20.29,2,0.158206,0.020789
42,17.46,2,0.145475,0.017889
161,21.50,4,0.162791,0.022029


In [82]:
X_train_scaled.columns.tolist()

['total_bill', 'size', 'tip_percentage', 'price_per_person']

In [83]:
# parameters: f_regression stats test, k = # of features (exercise says top 2)
f_selector = SelectKBest(f_regression, k=2)

# find the top 8 X's correlated with y
f_selector.fit(X_train_scaled, y_train)

# boolean mask of whether the column was selected or not. 
feature_mask = f_selector.get_support()

# get list of top K features. 
f_feature = X_train_scaled.iloc[:,feature_mask].columns.tolist()

In [84]:
feature_mask

array([ True, False, False,  True])

In [85]:
f_feature = X_train.iloc[:,feature_mask].columns.tolist()
f_feature

['total_bill', 'price_per_person']

In [86]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

lm = LinearRegression()
rfe = RFE(lm,2)
rfe.fit(X_train, y_train)
feature_mask_rfe = rfe.support_
rfe_feature = X_train.iloc[:,feature_mask].columns.tolist()

In [87]:
rfe_feature

['total_bill', 'price_per_person']

### **e. Use all the other numeric features to predict tip percentage. Use select k best and recursive feature elimination to select the top 2 features.**
    - What are they?
   


In [88]:
X_train, y_train, X_validate, y_validate, X_test, y_test = train_validate_test(tips, 'tip_percentage' )
X_train.drop(columns=object_cols,inplace=True)

In [89]:
f_selector = SelectKBest(f_regression, k=2)
f_selector.fit(X_train, y_train)
feature_mask = f_selector.get_support()

f_feature = X_train.iloc[:,feature_mask].columns.tolist()
f_feature

['total_bill', 'tip']

In [90]:
lm = LinearRegression()
rfe = RFE(lm,2)
rfe.fit(X_train, y_train)
feature_mask_rfe = rfe.support_
rfe_feature = X_train.iloc[:,feature_mask].columns.tolist()
rfe_feature

['total_bill', 'tip']

### **f. Why do you think select k best and recursive feature elimination might give different answers for the top features?**
    - Does this change as you change the number of features your are selecting?


- Many features are dependent upon one another 

**2. Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.**



In [91]:
def select_kbest(X, y, k):
    f_selector = SelectKBest(f_regression, k)
    f_selector.fit(X, y)
    feature_mask = f_selector.get_support()

    f_feature = X.iloc[:,feature_mask].columns.tolist()
    return f_feature

In [92]:
select_kbest(X_train, y_train, 2)

['total_bill', 'tip']

**3. Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.**



In [93]:
def rfe(X,y,k):
    lm = LinearRegression()
    rfe = RFE(lm,k)
    rfe.fit(X, y)
    feature_mask_rfe = rfe.support_
    rfe_feature = X.iloc[:,feature_mask_rfe].columns.tolist()
    return rfe_feature

In [94]:
rfe(X_train, y_train, 2)

['total_bill', 'tip']

**4. Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).**

In [96]:
swiss = pydataset.data('swiss')
swiss.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [97]:
X_train, y_train, X_validate, y_validate, X_test, y_test = train_validate_test(swiss, 'Fertility' )

In [98]:
select_kbest(X_train,y_train,3)

['Examination', 'Catholic', 'Infant.Mortality']

In [99]:
rfe(X_train,y_train,3)

['Agriculture', 'Examination', 'Infant.Mortality']